In [1]:
import torch
import torch.nn as nn
import random
import string
import sys
import unidecode
from torch.utils.tensorboard import SummaryWriter

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_characters = string.printable
n_characters = len(all_characters)

print(n_characters, all_characters)

100 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



In [3]:
import pandas

file1 = pandas.read_csv('../input/indian-male-baby-names/Indian-Male-Names.csv')

file1.head()

,name,gender,race
0,barjraj,m,indian
1,ramdin verma,m,indian
2,sharat chandran,m,indian
3,birender mandal,m,indian
4,amit,m,indian


In [4]:
all_names = [str(x).split(" ")[0] for x in file1["name"].values]
# we only need first names
all_names[:5]

['barjraj', 'ramdin', 'sharat', 'birender', 'amit']

In [5]:
file2 = open("names_parsed.txt", 'w+')
file2.write("\n".join(all_names))

import unidecode
file = unidecode.unidecode(open("names_parsed.txt", "r+").read())

file[:5]

'barjr'

In [6]:
class myRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(myRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden, cell, test=False):
        out = self.embed(x)
        if (test):
            print("output: ", out)
            print("output unsqueeze 1: ", out.unsqueeze(1))
            print("hidden : ", hidden)
            print("cell : ", cell)
        out, ( hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        
        return out, (hidden, cell)
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device=device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device=device)
        return hidden, cell

### Testing

In [7]:
# input_size = 10
# output_size = 2
# hidden_size = 2
# num_layers = 2
# batch_size = 2

In [8]:
# rnn = myRNN(input_size, output_size, hidden_size, num_layers)

In [9]:
# hidden,cell = rnn.init_hidden(batch_size)
# hidden.shape, cell.shape

In [10]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# x = torch.tensor([1,2]).to(device=device)

# x is ultimately a tensor of index

In [12]:
# rnn.forward(x, hidden, cell)

# it is working but dont know how

https://www.youtube.com/watch?v=WujVlF_6h5A&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=33

### testing over

In [13]:
len(all_characters)

100

In [14]:
class myGenerator:
    def __init__(self):
        self.chunk_len = 250
        self.hidden_size = 256
        self.num_layers = 2
        self.lr = 0.003
        self.num_epochs = 201
        self.print_every = 40
        self.batch_size = 1
    
    def char_tensor(self, str_input):
        #converts string input to tensor 
        c_tensor = torch.zeros(len(str_input)).long()
        
        for c in range(len(str_input)):
            try:
                c_tensor[c] = all_characters.index(str_input[c])
            except Exception as e:
                print(e)
                print(f" error by {str_input}")
        
        return c_tensor

    def get_random_batch(self):
        start_idx = random.randint(0, len(file) - self.chunk_len)
        end_idx = start_idx + self.chunk_len + 1
        
        text_str = file[start_idx:end_idx]
        text_input = torch.zeros(self.batch_size, self.chunk_len)
        text_target = torch.zeros(self.batch_size, self.chunk_len)
        
        for i in range(self.batch_size):
            text_input[i, :] = self.char_tensor(text_str[:-1])
            text_target[i, :] = self.char_tensor(text_str[1:])
        
        return text_input.long(), text_target.long()
    
    def generate(self, initial_str = "A", predict_len = 100, temprature=0.85):
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.char_tensor(initial_str)
        
        predicted = initial_str
        
        for p in range(len(initial_str) - 1):
            _, (hidden,cell) = self.rnn(initial_input[p].view(1).to(device=device), hidden, cell)
        
        last_char = initial_input[-1]
        
        for p in range(predict_len):
            output, (hidden, cell) = self.rnn(last_char.view(1).to(device=device), hidden, cell)
            output_dist = output.data.view(-1).div(temprature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]
            predicted_char = all_characters[top_char]
            predicted += predicted_char
            last_char = self.char_tensor(predicted_char)
            
        return predicted
            
    def train(self):
        #input_size, output_size, hidden_size, num_layers
        self.rnn = myRNN(n_characters, self.hidden_size, self.num_layers, n_characters).to(device=device)
        optimizer = torch.optim.Adam(self.rnn.parameters(),lr = self.lr)
        
        loss_criterion = nn.CrossEntropyLoss()
        writer = SummaryWriter(f'runs/names0')
        
        print("start training")
        
        for epoch in range(1,self.num_epochs+1):
            inp, target = self.get_random_batch()
            
            inp = inp.to(device=device)
            target = target.to(device=device)
            
            optimizer.zero_grad()
            
            hidden, cell = self.rnn.init_hidden(batch_size= self.batch_size)
            loss = 0
            
            for c in range(self.chunk_len):
                output, (hidden, cell) = self.rnn(inp[:,c], hidden, cell)
                loss += loss_criterion(output, target[:,c])
#             out , (hidden1, cell1) = self.rnn(inp)
#             loss = loss_criterion(out, target)
            
            loss.backward()
            
            optimizer.step()
            loss = loss.item() / self.chunk_len
            
            if epoch % self.print_every == 0:
                print(f"loss for epoch {epoch+1} : {loss}")
                print(self.generate())
            
            writer.add_scalar('Training loss', loss, global_step=epoch)
            
            
        
        
genames = myGenerator()
genames.train()        

start training
loss for epoch 41 : 2.262888427734375
ASL<\5~D HRYiyj
arda
thinpar
maral
ranesh
soyieein
irkoeph
rakaer
namdun
rayyak
powan
ukit
mavdnd
dba
loss for epoch 81 : 1.837861572265625
A}*>Cnkesh
sraun
ahral
dunak
kohandet
ajtat
nanmin
rrinkot
maley
rael
arit
surikral
ennish
babharak
s
loss for epoch 121 : 1.884819091796875
A6z(*?):Q;@av
keepar
gaijen
karaeen
avikan
ranja
aret
panmu
ganshi
inish
deran
sani
pil
ravendan
piro
loss for epoch 161 : 1.854039794921875
AB^?3LL
mohd
sokesh
mohy
narad
daraj
jubhari
baresh
deran
viraj
majar
nash
iril
parbeit
dirid
arjaj
j
loss for epoch 201 : 2.0109595947265624
A76^ZH$`f^Oamash
ammat
chahendner
saichan
ghawakrama
tanu
sacnu
razirveer
mohd
sushal
saaturath
vishn
